![ine-divider](https://user-images.githubusercontent.com/7065401/92672068-398e8080-f2ee-11ea-82d6-ad53f7feb5c0.png)
<hr>

### MySQL and MariaDB for Python Developers
# Interacting with Python versus MySQL data types

In this project, you will explore the interfaces between Python data types and MySQL data types.

You will need access to a MySQL installation where you have superuser permissions. If you do not have such access elsewhere, installing to your personal workstation is a good idea.  Alternately, you might wish to use a Docker container for a self-contained installation.  See ` https://hub.docker.com/_/mysql` for details on that option.

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Part 1

**Dynamically guessing good schema**

Python variables and attributes are dynamically typed, but each individual value has a strict type.  We sometimes are presented with collections of Python collections, and would like to determine good data layout for this data in MySQL automatically.

For this task, assume that your Python data is in the form of an iterable of namedtuples.  After analyzing data input, your function should product a MySQL SQL statement to create a proposed table.  Obviously, emphasizing that this inference is a guess is important, since future Pyton data produced in the same application may not be compatible with the schema. In use, the function should operate in a manner similar to the below:

```python
>>> print(infer_schema(list_of_named_tuples))
CREATE TABLE my_records (
    a SMALLINT, 
    b BIGINT DEFAULT NULL,
    c DECIMAL(40,25),
    d REAL DEFAULT NULL,
    e TEXT
);
```

At the least, the proposed schema should be compatible with the data actually encountered.  If you decide it is not possible to unify the data in a single data definition, you should raise an appropriate exception.  Several sample datasets are provided for you to test against.  You should expand these for more robust testing, especially to consider additional edge cases.

In [1]:
from samples import data1, data2, data3, data4, data5, data6
# For example...
data1

[Numbers1(a=1.23, b=Decimal('1.15573'), c=1000000000000, d=2, e=Fraction(22, 7)),
 Numbers1(a=4.56, b=Decimal('1.155727349790921717935726'), c=-22, d=5, e=Fraction(1, 3)),
 Numbers1(a=7.89, b=Decimal('1.0'), c=56, d=9, e=Fraction(5, 1))]

**A possible solution**

As in other projects, what is suggested is code that might satisfy the task.  A more fleshed out version would certainly address additional special cases and perform more sophisticated inference.

In [2]:
def unify_datatype(vals):
    # Floating point?
    if all(isinstance(v, float) for v in vals):
        # Python is 64-bit, so this is safest assumption
        return "DOUBLE PRECISION"
    
    # Integer?
    if all(isinstance(v, int) for v in vals):
        # Python has unlimited length ints, so have to guess length
        size = max(abs(v) for v in vals).bit_length()
        if size <= 16:
            return "SMALLINT"
        elif size <= 32:
            return "INTEGER"
        elif size <= 64:
            return "BIGINT"
        else:
            from math import log10
            return f"DECIMAL({int(1+log10(2**size))})"

    # Decimal?
    from decimal import Decimal
    if all(isinstance(v, Decimal) for v in vals):
        # The tricky part is how many digits after decimal point
        # ... the "context" of decimal module is weird!
        to_right = 0
        to_left = 1
        for v in vals:
            v_str = str(v)
            to_right = max(to_right, v_str[::-1].find('.'))
            to_left = max(to_left, v_str.find('.'))
        to_right += 1  # Need one more decimal point than calculated
        return f"DECIMAL({to_left+to_right}, {to_right})"
    
    # If it is string, is is currency? (for now, only know dollars)
    if all(isinstance(v, str) and v[0] == '$' for v in vals):
        return "DECIMAL(10, 2)"
    
    # MySQL does not have a Fraction/Rational type. 
    # What to do with Python Fractions?
    # Co-opt the POINT data type with X/Y as numerator/denominator?
    # Here we just convert to a float though
    from fractions import Fraction
    if all(isinstance(v, Fraction) for v in vals):
        return "DOUBLE PRECISION"  
    
    # If nothing else can be found, use the raw Python repr for the datatype
    return "TEXT"

In [3]:
def infer_schema_nt(dataset):
    # May have a lazy iterator, concretize (perhaps optimize this later)
    dataset = list(dataset)
    # Require at least two rows to infer datatype
    if len(dataset) < 2:
        raise ValueError("At least two rows are required for inference")
    # Check if they are all namedtuples (probably)
    if not all(isinstance(t, tuple) and hasattr(t, '_fields') for t in dataset):
        raise ValueError("The dataset does not appear to be exclusively namedtuples")
    # Check that namedtuples have same type
    if not len({o.__class__.__name__ for o in dataset}) == 1:
        raise ValueError("The dataset has namedtuples of varying types")

    tablename = dataset[0].__class__.__name__
    fields = dataset[0]._fields
    types = {}

    # Can we find a good type for each column?
    for n, col in enumerate(fields):
        colvals = [row[n] for row in dataset]
        if not (coldef := unify_datatype(colvals)):
            raise ValueError(f"Could not find unified datatype for column {col}")
        types[col] = coldef

    # Format the DDL SQL command
    sql = [f"CREATE TABLE {tablename} ("]
    for col, typ in types.items():
        sql.append(f"    {col} {typ},")
    sql[-1] = sql[-1].rstrip(',')
    sql.append(");")
    return  "\n".join(sql)

In [4]:
for data in "data1 data2 data3 data4 data5 data6".split():
    print(data)
    data = eval(data)
    try:
        print(infer_schema_nt(data))
    except ValueError as err:
        print(err)
    print('-'*50)

data1
CREATE TABLE Numbers1 (
    a DOUBLE PRECISION,
    b DECIMAL(26, 25),
    c BIGINT,
    d SMALLINT,
    e DOUBLE PRECISION
);
--------------------------------------------------
data2
The dataset has namedtuples of varying types
--------------------------------------------------
data3
CREATE TABLE Numbers3 (
    a DECIMAL(10, 2),
    b DECIMAL(26, 25),
    c BIGINT,
    d SMALLINT,
    e DOUBLE PRECISION
);
--------------------------------------------------
data4
The dataset does not appear to be exclusively namedtuples
--------------------------------------------------
data5
At least two rows are required for inference
--------------------------------------------------
data6
CREATE TABLE Numbers1 (
    a DOUBLE PRECISION,
    b DECIMAL(26, 25),
    c DECIMAL(21),
    d SMALLINT,
    e DOUBLE PRECISION
);
--------------------------------------------------


![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Part 2

**Working with other Python collections**

No solution is provided here, but you should be able to re-use most of the work with namedtuples.  Consider how you would need to change the table inference if you are given an iterable of dictionaries? What about an iterable of data classes? What about an iterable of plain lists or tuples. What about and iterable of custom Python objects with various attributes.

What would be reasonable exception checking if you wished to use a iterable of heterogeneous Python "data objects"? In some ways, it might be reasonable to consider a namedtuple data class, or dictionary "morally equivalent" from the point-of-view of a PostgreSQL table.  What limits are likely to apply.

The solution provided to part 1 did not consider NULLable columns. It might be reasonable to look for Python `None` values in the data set and use that as guidance for being NULLable.  Moreover, if dictionaries or other mappings are the source data, it *might* (or might not) be appropriate to treat a missing key as a NULL value.

**Notes on a solution**

All the logic in the function `unify_datatype()` should remain identical when working with other "data objects" in Python.  The aspects that will differ when working with objects other than namedtuples are how to identify the column names, how to validate compatible structures, how to extract the data corresponding to one column, and so on.  

Once you find "all the objects that might go in this column", you can use the identical `unify_datatype()` function.  It is worthwhile to think about how to make that function more robust or general, of course.

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)